# Final_Assessment (Lesson/Unit 8)

In [2]:
# 17.1 Bipartite ثنائي الأطراف

#
# Write a function, `bipartite` that
# takes as input a graph, `G` and tries
# to divide G into two sets where
# there are no edges between elements of the
# the same set - only between elements in
# different sets.
# If two sets exists, return one of them
# or `None` otherwise
# Assume G is connected
#

def bipartite(G):
	checked = {}

	def _iter_check(node, side):
		if node in checked:
			return
		checked[node] = side
		for neighbor in G[node]:
			_iter_check(neighbor, not side)

	for node in G:
		_iter_check(node, True)

	def _valid(subset):
		for node in subset:
			for neighbor in G[node]:
				if neighbor in subset:
					return False
		return True

	left_set  = set(filter(lambda x: checked[x], checked))
	right_set = set(G.keys()) - left_set

	if _valid(left_set) and _valid(right_set):
		return left_set

########
#
# Test

def make_link(G, node1, node2, weight=1):
	if node1 not in G:
		G[node1] = {}
	(G[node1])[node2] = weight
	if node2 not in G:
		G[node2] = {}
	(G[node2])[node1] = weight
	return G

def test():
	edges = [(1, 2), (2, 3), (1, 4), (2, 5),
			 (3, 8), (5, 6)]
	G = {}
	for n1, n2 in edges:
		make_link(G, n1, n2)
	g1 = bipartite(G)
	assert (g1 == set([1, 3, 5]) or
			g1 == set([2, 4, 6, 8]))
	edges = [(1, 2), (1, 3), (2, 3)]
	G = {}
	for n1, n2 in edges:
		make_link(G, n1, n2)
	g1 = bipartite(G)
	assert g1 == None

if __name__ == '__main__':
	test()
	print("Test passes")

Test passes


In [3]:
# 17.2 Feel the Love

#
# Take a weighted graph representing a social network where the weight
# between two nodes is the "love" between them.  In this "feel the
# love of a path" problem, we want to find the best path from node `i`
# and node `j` where the score for a path is the maximum love of an
# edge on this path. If there is no path from `i` to `j` return
# `None`.  The returned path doesn't need to be simple, ie it can
# contain cycles or repeated vertices.
#
# Devise and implement an algorithm for this problem.
#

import heapq
from collections import defaultdict

def feel_the_love(G, i, j):
	# return a path (a list of nodes) between `i` and `j`,
	# with `i` as the first node and `j` as the last node,
	# or None if no path exists
	path = dijkstra_path(G, i)
	if not j in path:
		return None

	node_a, node_b = max_weight_edge(G, i)
	path_a = path[node_a]
	path_b = (dijkstra_path(G, node_b))[j]

	return path_a + path_b

def max_weight_edge(G, i):
	max_so_far = -float('inf')
	edge       = None
	reachable  = dijkstra_path(G, i)
	for node in G:
		for neighbor in G[node]:
			if (G[node])[neighbor] > max_so_far and node in reachable:
				max_so_far = (G[node])[neighbor]
				edge = node, neighbor

	return edge

def dijkstra_path(HG, v):
	dist_so_far = {v: 0}
	final_dist = {}
	final_path = defaultdict(list)
	heap = [(0, v)]
	while dist_so_far:
		(w, k) = heapq.heappop(heap)
		if k in final_dist or (k in dist_so_far and w > dist_so_far[k]):
			continue
		else:
			del dist_so_far[k]
			final_dist[k] = w
		for neighbor in [nb for nb in HG[k] if nb not in final_dist]:
			nw = final_dist[k]+ HG[k][neighbor]
			final_path[neighbor] = final_path[k] + [k]

			if neighbor not in dist_so_far or nw < dist_so_far[neighbor]:
				dist_so_far[neighbor] = nw
				heapq.heappush(heap, (nw, neighbor))

	for node in final_path:
		final_path[node] += [node]
	return final_path

#########
#
# Test

def score_of_path(G, path):
	max_love = -float('inf')
	for n1, n2 in zip(path[:-1], path[1:]):
		love = G[n1][n2]
		if love > max_love:
			max_love = love
	return max_love

def test():
	G = {'a':{'c':1},
		 'b':{'c':1},
		 'c':{'a':1, 'b':1, 'e':1, 'd':1},
		 'e':{'c':1, 'd':2},
		 'd':{'e':2, 'c':1},
		 'f':{}}
	path = feel_the_love(G, 'a', 'b')
	assert score_of_path(G, path) == 2

	path = feel_the_love(G, 'a', 'f')
	assert path == None

if __name__ == '__main__':
	test()
	print("Test passes")

Test passes


In [4]:
# 17.3 Weighted Graph

#
# In lecture, we took the bipartite Marvel graph,
# where edges went between characters and the comics
# books they appeared in, and created a weighted graph
# with edges between characters where the weight was the
# number of comic books in which they both appeared.
#
# In this assignment, determine the weights between
# comic book characters by giving the probability
# that a randomly chosen comic book containing one of
# the characters will also contain the other
#

import itertools
from collections import defaultdict
import pickle  # Use pickle instead of cPickle for Python 3

# Open files in binary mode ('rb')
marvel = pickle.load(open("smallG.pkl", "rb"))
characters = pickle.load(open("smallChr.pkl", "rb"))

def create_weighted_graph(bipartiteG, characters):
    G = defaultdict(dict)
    for char_a, char_b in itertools.combinations(characters, 2):
        a_books = set(bipartiteG[char_a])
        b_books = set(bipartiteG[char_b])

        inter_book_num = float(len(a_books.intersection(b_books)))
        if inter_book_num == 0:
            continue

        prob = inter_book_num / (len(a_books) + len(b_books) - inter_book_num)
        G[char_a][char_b] = prob
        G[char_b][char_a] = prob

    return G

######
#
# Test

def test():
    bipartiteG = {
        'charA': {'comicB':1, 'comicC':1},
        'charB': {'comicB':1, 'comicD':1},
        'charC': {'comicD':1},
        'comicB': {'charA':1, 'charB':1},
        'comicC': {'charA':1},
        'comicD': {'charC':1, 'charB':1}
    }
    G = create_weighted_graph(bipartiteG, ['charA', 'charB', 'charC'])
    assert G['charA']['charB'] == 1.0 / 3
    assert G['charA'].get('charA') == None
    assert G['charA'].get('charC') == None

def test2():
    G = create_weighted_graph(marvel, characters)

if __name__ == '__main__':
    test()
    test2()
    print("Test passes")

Test passes


In [19]:
# 17.4 Finding the Best Flight العثور على أفضل رحلة

#
# In many path-finding applications, a natural scoring function is
# "lexicographic ordering".  That is, there is one attribute of the
# path (say cost) that is the most important thing to minimize.
# However, all things being equal, if you have two paths with the same
# cost, you might prefer one with a shorter total flight time.
#
# A recent example of lexicographic ordering: In the Olympics Medal
# Tracker website, countries are sorted from most total medals to
# least total medals.  If two countries have the same number of total
# medals, the one with more gold medals is listed first.  If they have
# the same total medals and the same number of gold medals, the one
# with more silver medals is listed first.
#
# We want you to take the list of flights, given below, and create a
# graph.  Then, write a modified Dijkstra's algorithm to find the best
# combination of flights to get between two cities, where flights `x`
# is better than flights `y` if `x` has lower cost *or* if they are
# tied in cost, `x` has shorter total flight time.
#
# Concretely, to get from Broome to Fitroy Crossing,
# flights [530, 112] are better than flights [526, 622]
# because, since they both cost 110, the first flights are
# shorter - 5 hours and 52 minutes compared to
# 6 hours and 23 minutes. There maybe be even better flights, but
# you'll have to search the graph to find them.
#

import heapq

def make_link(G, num, city1, city2, flight_duration,
              departure, arrival, cost):  
    # create links between two cities, links are flights.
    if city1 not in G:
        G[city1] = {}
    if city2 not in G[city1]:
        G[city1][city2] = {}
    G[city1][city2][num] = (cost, departure, arrival, flight_duration)
    return G

def create_connection_graph(data):
    G = {}
    for flight in data:
        (num, city1, city2, dept, arrv, cost) = flight
        dept = int(dept.replace(':', '', 1))
        deptmin = (dept/100)*60  + dept%100
        arrv = int(arrv.replace(':', '', 1))
        arrvmin  = (arrv/100)*60 + arrv%100
        durn = arrvmin - deptmin
        make_link(G, num, city1, city2, durn, deptmin, arrvmin, cost)            
    return G
                           
def find_best_flights(flights, origin, destination):
    # A dijkstra implementation
    G = create_connection_graph(flights)
    heap = []
    heapq.heappush(heap, (0,0,0,[],[origin]))
    paths = {origin:[origin]}
    while len(heap) != 0:
        (fcost, fdurn, farrv, flights, path) = heapq.heappop(heap)
        w = path[-1]
        if w == destination:
            return flights
        for x in G[w]:
            for flight in G[w][x]:
                (cost, dept, arrv, durn) = G[w][x][flight]
                if dept > farrv:
                    new_cost = cost + fcost
                    if fdurn == 0:
                        new_durn = arrv - dept
                    else:
                        new_durn = fdurn + (arrv - farrv)
                    new_path = path + [x]
                    new_flights = flights + [flight]
                    heapq.heappush(heap, (new_cost, new_durn, arrv,
                                          new_flights, new_path))
    return None

#
# Here is a fictious flight schedule that is roughly based on routes
# flown by Skipper, a regional airline in Australia
# (http://www.skippers.com.au/).
#
# Each tuple contains six items:
#   Flight Number, Origin, Destination, Departure Time, Arrival Time, Cost
# (Don't worry about any time zone issues; assume everything happens
# in the same time zone)
# Also note that overnight layovers are not allowed.
#
all_flights = [(523, 'Broome', 'Derby', '07:17', '08:57', 60),
			(526, 'Broome', 'Derby', '08:41', '10:30', 50),
			(527, 'Broome', 'Derby', '11:46', '13:24', 200),
			(530, 'Broome', 'Derby', '14:23', '15:59', 50),
			(540, 'Broome', 'Derby', '17:49', '19:40', 50),
			(546, 'Broome', 'Derby', '20:34', '22:09', 20),
			(547, 'Broome', 'Perth', '06:41', '08:44', 30),
			(549, 'Broome', 'Perth', '17:16', '19:18', 100),
			(559, 'Carnarvon', 'Geraldton', '09:05', '10:57', 50),
			(561, 'Carnarvon', 'Geraldton', '11:14', '13:03', 30),
			(578, 'Carnarvon', 'Geraldton', '14:56', '16:48', 150),
			(582, 'Carnarvon', 'Geraldton', '17:05', '18:46', 50),
			(598, 'Carnarvon', 'Geraldton', '22:08', '23:49', 20),
			(599, 'Carnarvon', 'Perth', '07:04', '09:46', 200),
			(100, 'Carnarvon', 'Perth', '10:53', '13:38', 60),
			(604, 'Carnarvon', 'Perth', '14:50', '17:16', 200),
			(612, 'Carnarvon', 'Perth', '19:54', '22:38', 50),
			(107, 'Derby', 'Broome', '08:44', '10:36', 160),
			(108, 'Derby', 'Broome', '21:18', '23:04', 30),
			(622, 'Derby', 'Fitzroy Crossing', '13:59', '15:04', 60),
			(112, 'Derby', 'Fitzroy Crossing', '19:24', '20:15', 60),
			(113, 'Derby', 'Geraldton', '07:00', '08:10', 20),
			(115, 'Derby', 'Geraldton', '10:00', '11:07', 200),
			(118, 'Derby', 'Geraldton', '13:24', '14:31', 50),
			(121, 'Derby', 'Geraldton', '14:41', '15:52', 50),
			(122, 'Derby', 'Geraldton', '17:05', '18:09', 60),
			(635, 'Derby', 'Geraldton', '18:59', '20:18', 60),
			(638, 'Fitzroy Crossing', 'Derby', '09:18', '10:08', 50),
			(131, 'Fitzroy Crossing', 'Derby', '13:59', '14:51', 160),
			(226, 'Fitzroy Crossing', 'Derby', '14:34', '15:34', 110),
			(139, 'Fitzroy Crossing', 'Derby', '18:43', '19:36', 50),
			(654, 'Fitzroy Crossing', 'Halls Creek', '07:55', '09:48', 180),
			(143, 'Fitzroy Crossing', 'Halls Creek', '09:45', '11:39', 20),
			(280, 'Fitzroy Crossing', 'Halls Creek', '15:10', '17:07', 110),
			(660, 'Fitzroy Crossing', 'Halls Creek', '18:41', '20:24', 30),
			(661, 'Fitzroy Crossing', 'Halls Creek', '20:35', '22:19', 200),
			(663, 'Geraldton', 'Carnarvon', '08:30', '10:24', 30),
			(152, 'Geraldton', 'Carnarvon', '12:52', '14:42', 50),
			(153, 'Geraldton', 'Carnarvon', '15:24', '17:15', 30),
			(154, 'Geraldton', 'Carnarvon', '18:07', '19:53', 180),
			(671, 'Geraldton', 'Derby', '06:01', '07:10', 120),
			(676, 'Geraldton', 'Derby', '10:46', '12:09', 20),
			(165, 'Geraldton', 'Derby', '11:29', '12:45', 30),
			(683, 'Geraldton', 'Derby', '14:17', '15:23', 50),
			(174, 'Geraldton', 'Derby', '16:45', '17:58', 180),
			(175, 'Geraldton', 'Derby', '18:31', '19:47', 20),
			(179, 'Halls Creek', 'Fitzroy Crossing', '06:32', '08:22', 200),
			(187, 'Halls Creek', 'Fitzroy Crossing', '13:19', '15:03', 200),
			(702, 'Halls Creek', 'Fitzroy Crossing', '14:04', '15:45', 20),
			(192, 'Halls Creek', 'Fitzroy Crossing', '20:08', '21:59', 160),
			(195, 'Halls Creek', 'Kalbarri', '06:43', '09:01', 110),
			(709, 'Halls Creek', 'Kalbarri', '08:45', '11:04', 200),
			(199, 'Halls Creek', 'Kalbarri', '13:21', '15:39', 20),
			(209, 'Halls Creek', 'Kalbarri', '15:45', '18:01', 100),
			(723, 'Halls Creek', 'Kalbarri', '16:04', '18:10', 50),
			(724, 'Halls Creek', 'Kalbarri', '19:52', '22:07', 160),
			(216, 'Kalbarri', 'Halls Creek', '06:15', '08:34', 100),
			(217, 'Kalbarri', 'Halls Creek', '14:57', '17:14', 200),
			(730, 'Kalbarri', 'Halls Creek', '21:05', '23:24', 20),
			(731, 'Kalbarri', 'Perth', '06:18', '08:50', 50),
			(734, 'Kalbarri', 'Perth', '12:23', '14:59', 120),
			(735, 'Kalbarri', 'Perth', '12:59', '15:19', 30),
			(738, 'Kalbarri', 'Perth', '18:41', '21:10', 60),
			(739, 'Kalbarri', 'Perth', '19:42', '22:18', 60),
			(740, 'Laverton', 'Leonora', '07:39', '08:53', 180),
			(745, 'Laverton', 'Leonora', '12:20', '13:32', 20),
			(748, 'Laverton', 'Leonora', '13:44', '15:08', 30),
			(751, 'Laverton', 'Leonora', '18:00', '19:11', 200),
			(240, 'Laverton', 'Leonora', '20:34', '21:40', 110),
			(754, 'Laverton', 'Perth', '07:21', '08:21', 180),
			(247, 'Laverton', 'Perth', '20:11', '21:22', 160),
			(248, 'Leinster', 'Perth', '08:37', '11:16', 180),
			(249, 'Leinster', 'Perth', '13:44', '16:12', 110),
			(763, 'Leinster', 'Perth', '16:29', '19:06', 160),
			(765, 'Leinster', 'Perth', '19:17', '21:47', 20),
			(981, 'Leinster', 'Wiluna', '10:51', '13:03', 200),
			(770, 'Leinster', 'Wiluna', '16:02', '18:17', 50),
			(259, 'Leinster', 'Wiluna', '19:44', '22:09', 60),
			(772, 'Leonora', 'Laverton', '10:39', '11:59', 110),
			(987, 'Leonora', 'Laverton', '15:56', '17:13', 110),
			(264, 'Leonora', 'Laverton', '21:39', '22:48', 200),
			(779, 'Leonora', 'Perth', '10:29', '11:59', 50),
			(780, 'Leonora', 'Perth', '11:26', '12:58', 50),
			(783, 'Leonora', 'Perth', '19:48', '21:25', 30),
			(278, 'Meekatharra', 'Mt Magnet', '07:40', '08:42', 60),
			(792, 'Meekatharra', 'Mt Magnet', '08:35', '09:35', 60),
			(793, 'Meekatharra', 'Mt Magnet', '11:50', '12:44', 110),
			(796, 'Meekatharra', 'Mt Magnet', '14:32', '15:26', 30),
			(798, 'Meekatharra', 'Mt Magnet', '16:56', '17:52', 160),
			(288, 'Meekatharra', 'Mt Magnet', '19:38', '20:27', 60),
			(289, 'Meekatharra', 'Perth', '08:12', '09:28', 50),
			(803, 'Meekatharra', 'Perth', '09:12', '10:25', 30),
			(805, 'Meekatharra', 'Perth', '12:10', '13:16', 50),
			(298, 'Meekatharra', 'Perth', '13:33', '14:40', 50),
			(391, 'Meekatharra', 'Perth', '16:45', '17:50', 30),
			(815, 'Meekatharra', 'Perth', '20:17', '21:29', 110),
			(817, 'Monkey Mia', 'Perth', '08:26', '10:51', 20),
			(393, 'Monkey Mia', 'Perth', '13:12', '15:51', 30),
			(825, 'Monkey Mia', 'Perth', '21:01', '23:37', 180),
			(314, 'Mt Magnet', 'Meekatharra', '06:29', '07:30', 30),
			(827, 'Mt Magnet', 'Meekatharra', '08:56', '10:00', 50),
			(829, 'Mt Magnet', 'Meekatharra', '13:09', '14:14', 30),
			(832, 'Mt Magnet', 'Meekatharra', '14:10', '15:09', 30),
			(833, 'Mt Magnet', 'Meekatharra', '17:39', '18:41', 180),
			(322, 'Mt Magnet', 'Meekatharra', '19:51', '20:55', 160),
			(333, 'Mt Magnet', 'Perth', '07:53', '08:38', 120),
			(846, 'Mt Magnet', 'Perth', '15:45', '16:29', 20),
			(967, 'Mt Magnet', 'Perth', '18:04', '18:49', 20),
			(336, 'Mt Magnet', 'Wiluna', '07:34', '09:08', 200),
			(338, 'Mt Magnet', 'Wiluna', '13:35', '15:17', 30),
			(856, 'Mt Magnet', 'Wiluna', '14:54', '16:27', 50),
			(345, 'Mt Magnet', 'Wiluna', '18:03', '19:35', 50),
			(859, 'Perth', 'Broome', '07:21', '09:14', 50),
			(348, 'Perth', 'Broome', '10:37', '12:46', 60),
			(349, 'Perth', 'Broome', '12:56', '14:57', 20),
			(350, 'Perth', 'Broome', '15:01', '17:11', 110),
			(356, 'Perth', 'Broome', '18:03', '20:03', 60),
			(364, 'Perth', 'Broome', '18:45', '20:54', 150),
			(880, 'Perth', 'Carnarvon', '07:39', '10:09', 50),
			(884, 'Perth', 'Carnarvon', '10:33', '13:11', 30),
			(374, 'Perth', 'Carnarvon', '12:04', '14:31', 50),
			(375, 'Perth', 'Carnarvon', '13:59', '16:32', 30),
			(378, 'Perth', 'Carnarvon', '17:04', '19:38', 50),
			(299, 'Perth', 'Carnarvon', '19:27', '22:09', 50),
			(383, 'Perth', 'Kalbarri', '06:41', '09:12', 120),
			(384, 'Perth', 'Kalbarri', '12:42', '15:03', 20),
			(898, 'Perth', 'Kalbarri', '19:13', '21:38', 30),
			(390, 'Perth', 'Laverton', '10:20', '11:23', 60),
			(321, 'Perth', 'Laverton', '14:08', '15:03', 60),
			(905, 'Perth', 'Laverton', '19:58', '20:53', 100),
			(395, 'Perth', 'Leinster', '06:59', '09:28', 200),
			(396, 'Perth', 'Leinster', '10:17', '12:48', 100),
			(401, 'Perth', 'Leinster', '14:24', '16:50', 50),
			(914, 'Perth', 'Leinster', '18:54', '21:34', 160),
			(404, 'Perth', 'Leonora', '11:03', '12:40', 30),
			(918, 'Perth', 'Leonora', '12:37', '14:17', 150),
			(408, 'Perth', 'Leonora', '20:42', '22:10', 100),
			(923, 'Perth', 'Meekatharra', '06:21', '07:35', 110),
			(927, 'Perth', 'Meekatharra', '10:25', '11:26', 20),
			(933, 'Perth', 'Meekatharra', '14:27', '15:24', 50),
			(934, 'Perth', 'Meekatharra', '17:49', '18:50', 200),
			(941, 'Perth', 'Meekatharra', '21:56', '23:08', 30),
			(430, 'Perth', 'Monkey Mia', '06:18', '08:48', 30),
			(943, 'Perth', 'Monkey Mia', '12:11', '14:48', 180),
			(432, 'Perth', 'Monkey Mia', '17:32', '20:13', 50),
			(433, 'Perth', 'Monkey Mia', '19:48', '22:23', 100),
			(947, 'Perth', 'Mt Magnet', '06:43', '07:23', 100),
			(948, 'Perth', 'Mt Magnet', '13:59', '14:54', 20),
			(954, 'Perth', 'Mt Magnet', '15:44', '16:26', 120),
			(955, 'Perth', 'Mt Magnet', '19:34', '20:26', 200),
			(475, 'Perth', 'Wiluna', '07:34', '09:57', 60),
			(959, 'Perth', 'Wiluna', '09:44', '12:22', 50),
			(455, 'Perth', 'Wiluna', '12:22', '14:45', 60),
			(969, 'Perth', 'Wiluna', '14:26', '16:59', 50),
			(458, 'Perth', 'Wiluna', '17:19', '19:38', 60),
			(459, 'Perth', 'Wiluna', '19:09', '21:35', 30),
			(461, 'Wiluna', 'Leinster', '07:54', '10:16', 20),
			(462, 'Wiluna', 'Leinster', '08:35', '10:50', 200),
			(463, 'Wiluna', 'Leinster', '11:50', '14:01', 200),
			(976, 'Wiluna', 'Leinster', '13:54', '16:15', 50),
			(469, 'Wiluna', 'Leinster', '17:24', '19:43', 30),
			(984, 'Wiluna', 'Leinster', '19:58', '22:13', 200),
			(847, 'Wiluna', 'Mt Magnet', '07:13', '08:42', 30),
			(478, 'Wiluna', 'Mt Magnet', '11:48', '13:14', 50),
			(993, 'Wiluna', 'Mt Magnet', '13:00', '14:27', 20),
			(483, 'Wiluna', 'Mt Magnet', '17:20', '18:57', 60),
			(422, 'Wiluna', 'Mt Magnet', '21:40', '23:21', 60),
			(494, 'Wiluna', 'Perth', '08:28', '11:07', 160),
			(253, 'Wiluna', 'Perth', '11:17', '13:41', 150),
			(498, 'Wiluna', 'Perth', '13:53', '16:13', 60),
			(501, 'Wiluna', 'Perth', '17:59', '20:27', 20),
			(505, 'Wiluna', 'Perth', '20:21', '22:41', 180)]


def test():
    flights = find_best_flights(all_flights, 'Mt Magnet', 'Fitzroy Crossing')
    assert flights == [314, 803, 348, 530, 112]

    flights = find_best_flights(all_flights, 'Leonora', 'Fitzroy Crossing')
    assert flights == None

    flights = find_best_flights(all_flights, 'Meekatharra', 'Wiluna')
    assert flights == [391, 459]

if __name__ == '__main__':
	test()
	print("Test passes")

AssertionError: 

In [23]:
# 17.5 Constantly Connected متصل باستمرار

#
# Design and implement an algorithm that can preprocess a
# graph and then answer the question "is x connected to y in the
# graph" for any x and y in constant time Theta(1).
#

#
# `process_graph` will be called only once on each graph.  If you want,
# you can store whatever information you need for `is_connected` in
# global variables
#

global_G = {}

def process_graph(G):
	global global_G
	global_G = G
	for node in global_G:
		to_visit = global_G[node].keys()

		while to_visit:
			new_node = to_visit.pop()

			global_G[node][new_node] = 1
			global_G[new_node][node] = 1

			for x in global_G[new_node]:
				if x not in global_G[node]:
					to_visit += [x]

#
# When being graded, `is_connected` will be called
# many times so this routine needs to be quick
#
def is_connected(i, j):
	# your code here
	return i in global_G[j] or j in global_G[i]

#######
# Testing
#
def test():
	G = {'a':{'b':1},
		 'b':{'a':1},
		 'c':{'d':1},
		 'd':{'c':1},
		 'e':{}}
	process_graph(G)
	assert is_connected('a', 'b') == True
	assert is_connected('a', 'c') == False

	G = {'a':{'b':1, 'c':1},
		 'b':{'a':1},
		 'c':{'d':1, 'a':1},
		 'd':{'c':1},
		 'e':{}}
	process_graph(G)
	assert is_connected('a', 'b') == True
	assert is_connected('a', 'c') == True
	assert is_connected('a', 'e') == False

if __name__ == '__main__':
	test()
	print("Test passes")

AttributeError: 'dict_keys' object has no attribute 'pop'

In [25]:
# 17.6 Distance Oracle (I) أوراكل المسافة (I)

#
# In the shortest-path oracle described in Andrew Goldberg's
# interview, each node has a label, which is a list of some other
# nodes in the network and their distance to these nodes. These lists
# have the property that
#
#  (1) for any pair of nodes (x,y) in the network, their lists will
#  have at least one node z in common
#
#  (2) the shortest path from x to y will go through z.
#
# Given a graph G that is a balanced binary tree, preprocess the graph to
# create such labels for each node.  Note that the size of the list in
# each label should not be larger than log n for a graph of size n.
#

#
# create_labels takes in a balanced binary tree and the root element
# and returns a dictionary, mapping each node to its label
#
# a label is a dictionary mapping another node and the distance to
# that node
#
def create_labels(binarytreeG, root):
	# BFS for the binary tree, meanwhile labeling each node in each level
	labels = {root: {root: 0}}
	frontier = [root]
	while frontier:
		cparent = frontier.pop(0)
		for child in binarytreeG[cparent]:
			if child in labels:
				continue
			labels[child] = {child: 0}
			weight = binarytreeG[cparent][child]
			labels[child][cparent] = weight
			# make use of the labels already computed
			for ancestor in labels[cparent]:
				labels[child][ancestor] = weight + labels[cparent][ancestor]
			frontier += [child]
	return labels

#######
# Testing
#

def get_distances(G, labels):
	# labels = {a:{b: distance from a to b,
	#              c: distance from a to c}}
	# create a mapping of all distances for
	# all nodes
	distances = {}
	for start in G:
		# get all the labels for my starting node
		label_node = labels[start]
		s_distances = {}
		for destination in G:
			shortest = float('inf')
			# get all the labels for the destination node
			label_dest = labels[destination]
			# and then merge them together, saving the
			# shortest distance
			for intermediate_node, dist in label_node.iteritems():
				# see if intermediate_node is our destination
				# if it is we can stop - we know that is
				# the shortest path
				if intermediate_node == destination:
					shortest = dist
					break
				other_dist = label_dest.get(intermediate_node)
				if other_dist is None:
					continue
				if other_dist + dist < shortest:
					shortest = other_dist + dist
			s_distances[destination] = shortest
		distances[start] = s_distances
	return distances

def make_link(G, node1, node2, weight=1):
	if node1 not in G:
		G[node1] = {}
	(G[node1])[node2] = weight
	if node2 not in G:
		G[node2] = {}
	(G[node2])[node1] = weight
	return G

def test():
	edges = [(1, 2), (1, 3), (2, 4), (2, 5), (3, 6), (3, 7),
			 (4, 8), (4, 9), (5, 10), (5, 11), (6, 12), (6, 13)]
	tree = {}
	for n1, n2 in edges:
		make_link(tree, n1, n2)
	labels = create_labels(tree, 1)
	distances = get_distances(tree, labels)
	assert distances[1][2] == 1
	assert distances[1][4] == 2

if __name__ == '__main__':
	test()
	print("Test passes")

AttributeError: 'dict' object has no attribute 'iteritems'

In [27]:
# 17.7 Distance Oracle (II)

#
# This is the same problem as "Distance Oracle I" except that instead of
# only having to deal with binary trees, the assignment asks you to
# create labels for all tree graphs.
#
# In the shortest-path oracle described in Andrew Goldberg's
# interview, each node has a label, which is a list of some other
# nodes in the network and their distance to these nodes.  These lists
# have the property that
#
#  (1) for any pair of nodes (x,y) in the network, their lists will
#  have at least one node z in common
#
#  (2) the shortest path from x to y will go through z.
#
# Given a graph G that is a tree, preprocess the graph to
# create such labels for each node.  Note that the size of the list in
# each label should not be larger than log n for a graph of size n.
#

#
# create_labels takes in a tree and returns a dictionary, mapping each
# node to its label
#
# a label is a dictionary mapping another node and the distance to
# that node
#

def count_nodes(treeG, node):
	# count all sub-nodes including itself
	cnts = {}
	visited = {}
	cnts[node] = count_nodes_rec(treeG, node, cnts, visited)
	return cnts

def count_nodes_rec(treeG, node, cnts, visited):
	visited[node] = True
	frontier = [node]
	cnts[node] = 1
	for v in treeG[node]:
		if v not in visited:
			cnts[node] += count_nodes_rec(treeG, v, cnts, visited)
	return cnts[node]

def create_labels(treeG):
	# find center node via rotation
	def find_cen(treeG, tmt_root, cnts):
		if cnts[tmt_root] == 1:
			return tmt_root
		mcc, mc = max((cnts[v], v) for v in treeG[tmt_root] if v not in cens_nodes)
		# center node found!
		if cnts[tmt_root] - mcc >= mcc:
			return tmt_root
		# rotate 'tmt_root' to mc
		cnts[mc] += cnts[tmt_root] - mcc
		cnts[tmt_root] -= mcc
		return find_cen(treeG, mc, cnts)
	# recursively finding center node for each 'sub-tree'
	def label_tree(tmt_root):
		cen = find_cen(treeG, tmt_root, cnts)
		label_sub(cen)
		for child in treeG[cen]:
			if child not in cens_nodes:
				label_tree(child)
	# BFS routine for tagging each descendant node with its sub-center node
	def label_sub(sub_cen):
		if sub_cen not in labels:
			labels[sub_cen] = {}
		labels[sub_cen][sub_cen] = 0
		cens_nodes[sub_cen] = True
		frontier = [sub_cen]
		visited = {}
		while frontier:
			v = frontier.pop(0)
			for neighbor in treeG[v]:
				if neighbor not in visited and neighbor not in cens_nodes:
					visited[neighbor] = True
					frontier += [neighbor]
					if neighbor not in labels:
						labels[neighbor] = {neighbor: 0}
					labels[neighbor][sub_cen] = treeG[v][neighbor] + labels[v][sub_cen]
	cens_nodes = {}
	labels = {}
	tmt_root = iter(treeG).next()
	cnts = count_nodes(treeG, tmt_root)
	label_tree(tmt_root)
	return labels

#######
# Testing
#

def get_distances(G, labels):
	# labels = {a:{b: distance from a to b,
	#              c: distance from a to c}}
	# create a mapping of all distances for
	# all nodes
	distances = {}
	for start in G:
		# get all the labels for my starting node
		label_node = labels[start]
		s_distances = {}
		for destination in G:
			shortest = float('inf')
			# get all the labels for the destination node
			label_dest = labels[destination]
			# and then merge them together, saving the
			# shortest distance
			for intermediate_node, dist in label_node.iteritems():
				# see if intermediate_node is our destination
				# if it is we can stop - we know that is
				# the shortest path
				if intermediate_node == destination:
					shortest = dist
					break
				other_dist = label_dest.get(intermediate_node)
				if other_dist is None:
					continue
				if other_dist + dist < shortest:
					shortest = other_dist + dist
			s_distances[destination] = shortest
		distances[start] = s_distances
	return distances

def make_link(G, node1, node2, weight=1):
	if node1 not in G:
		G[node1] = {}
	(G[node1])[node2] = weight
	if node2 not in G:
		G[node2] = {}
	(G[node2])[node1] = weight
	return G

def test():
	edges = [(1, 2), (1, 3), (2, 4), (2, 5), (3, 6), (3, 7),
			 (4, 8), (4, 9), (5, 10), (5, 11), (6, 12), (6, 13)]
	tree = {}
	for n1, n2 in edges:
		make_link(tree, n1, n2)
	labels = create_labels(tree)
	if labels:
		distances = get_distances(tree, labels)
	assert distances[1][2] == 1
	assert distances[1][4] == 2
	assert distances[1][2] == 1
	assert distances[1][4] == 2
	assert distances[4][1] == 2
	assert distances[1][4] == 2
	assert distances[2][1] == 1
	assert distances[1][2] == 1
	assert distances[1][1] == 0
	assert distances[2][2] == 0
	assert distances[9][9] == 0
	assert distances[2][3] == 2
	assert distances[12][13] == 2
	assert distances[13][8] == 6
	assert distances[11][12] == 6
	assert distances[1][12] == 3

def test2():
	edges = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7),
			 (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13)]
	tree = {}
	for n1, n2 in edges:
		make_link(tree, n1, n2)
	labels = create_labels(tree)
	distances = get_distances(tree, labels)
	assert distances[1][2] == 1
	assert distances[1][3] == 2
	assert distances[1][13] == 12
	assert distances[6][1] == 5
	assert distances[6][13] == 7
	assert distances[8][3] == 5
	assert distances[10][4] == 6

if __name__ == '__main__':
	test()
	test2()
	print("Test passes")

AttributeError: 'dict_keyiterator' object has no attribute 'next'

In [31]:
# 17.8 Finding a Favor العثور على معروف

# Finding a Favor v2
#
# Each edge (u,v) in a social network has a weight p(u,v) that
# represents the probability that u would do a favor for v if asked.
# Note that p(v,u) != p(u,v), in general.
#
# Write a function that finds the right sequence of friends to maximize
# the probability that v1 will do a favor for v2.
#

#
# Provided are two standard versions of dijkstra's algorithm that were
# discussed in class. One uses a list and another uses a heap.
#
# You should manipulate the input graph, G, so that it works using
# the given implementations.  Based on G, you should decide which
# version (heap or list) you should use.
#

# code for heap can be found in the instructors comments below
from heap import *
from operator import itemgetter
from collections import defaultdict
from math import log, exp

def reform_graph(G):
	new_graph = defaultdict(dict)
	for node in G:
		for neighbor in G[node]:
			new_graph[node][neighbor] = log(G[node][neighbor]) * -1
	return new_graph

def maximize_probability_of_favor(G, v1, v2):
	# your code here
	# call either the heap or list version of dijkstra
	# and return the path from `v1` to `v2`
	# along with the probability that v1 will do a favor
	# for v2

	def _count_edges():
		return sum([len(G[v]) for v in G])

	G = reform_graph(G)

	# Theata(dijkstra_list) = Theata(n^2 + m) = Theata(n^2)
	# Theata(dijkstra_heap) = Theata(n * log(n) + m * log(n)) = Theata(m * log(n))
	node_num = len(G.keys())
	edge_num = _count_edges()

	if edge_num * log(node_num) <= node_num ** 2:
		dist_dict = dijkstra_heap(G, v1)
	else:
		dist_dict = dijkstra_list(G, v1)

	path = []
	node = v2
	while True:
		path += [node]
		if node == v1:
			break
		_, node = dist_dict[path[-1]]

	path = list(reversed(path))
	prob_log = dist_dict[v2][0] * -1

	return path, exp(prob_log)

#
# version of dijkstra implemented using a heap
#
# returns a dictionary mapping a node to the distance
# to that node and the parent
#
# Do not modify this code
#
def dijkstra_heap(G, a):
	# Distance to the input node is zero, and it has
	# no parent
	first_entry = (0, a, None)
	heap = [first_entry]
	# location keeps track of items in the heap
	# so that we can update their value later
	location = {first_entry:0}
	dist_so_far = {a:first_entry}
	final_dist = {}
	while len(dist_so_far) > 0:
		dist, node, parent = heappopmin(heap, location)
		# lock it down!
		final_dist[node] = (dist, parent)
		del dist_so_far[node]
		for x in G[node]:
			if x in final_dist:
				continue
			new_dist = G[node][x] + final_dist[node][0]
			new_entry = (new_dist, x, node)
			if x not in dist_so_far:
				# add to the heap
				insert_heap(heap, new_entry, location)
				dist_so_far[x] = new_entry
			elif new_entry < dist_so_far[x]:
				# update heap
				decrease_val(heap, location, dist_so_far[x], new_entry)
				dist_so_far[x] = new_entry
	return final_dist

#
# version of dijkstra implemented using a list
#
# returns a dictionary mapping a node to the distance
# to that node and the parent
#
# Do not modify this code
#
def dijkstra_list(G, a):
	dist_so_far = {a:(0, None)} #keep track of the parent node
	final_dist = {}
	while len(final_dist) < len(G):
		node, entry = min(dist_so_far.items(), key=itemgetter(1))
		# lock it down!
		final_dist[node] = entry
		del dist_so_far[node]
		for x in G[node]:
			if x in final_dist:
				continue
			new_dist = G[node][x] + final_dist[node][0]
			new_entry = (new_dist, node)
			if x not in dist_so_far:
				dist_so_far[x] = new_entry
			elif new_entry < dist_so_far[x]:
				dist_so_far[x] = new_entry
	return final_dist

##########
#
# Test

def test():
	G = {'a':{'b':.9, 'e':.5},
		 'b':{'c':.9},
		 'c':{'d':.01},
		 'd':{},
		 'e':{'f':.5},
		 'f':{'d':.5}}
	path, prob = maximize_probability_of_favor(G, 'a', 'd')
	assert path == ['a', 'e', 'f', 'd']
	assert abs(prob - .5 * .5 * .5) < 0.001

if __name__ == '__main__':
	test()
	print("Test passes")

ModuleNotFoundError: No module named 'heap'

In [ ]:
# Done (17)
# Lesson Rating
# You just completed (17)
# How was it? ()
# Suggested Improvements: 